In [34]:
import pandas as pd
import emoji
import re
import string
import spacy

nlp = spacy.load("es_core_news_sm")

from sklearn.feature_extraction.text import CountVectorizer

In [16]:
agrev = pd.read_csv('train_aggressiveness.csv')

In [17]:
all_filenames = ["comentarios1.csv", "comentarios2.csv", "comentarios3.csv"]

In [18]:
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv.to_csv("combined_csv.csv", index=False, encoding='utf-8-sig')

In [19]:
agrev.drop('Id', axis=1, inplace=True)

In [20]:
df = pd.concat([agrev, combined_csv])

In [21]:
df.head()

,Category,Text
0,0,Soy el Clint Eastwood de los Puentes de Madiso...
1,0,"Actualmente ya pasó de moda la pucha joto, aho..."
2,0,¿Es cierto esto? Y no me refiero a lo que dijo...
3,0,Vuela pega y esquiva... la neta está de la ver...
4,0,Mejor puto disfraz de la noche!!!! 👊👊👊Por terc...


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9300 entries, 0 to 205
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  9300 non-null   int64 
 1   Text      9300 non-null   object
dtypes: int64(1), object(1)
memory usage: 218.0+ KB


In [23]:
def clean_text(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('.*?¿', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    return text

In [24]:
def eliminar_stop_words(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop]
    return " ".join(token for token in tokens)

In [25]:
def lemmatize(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]
    return " ".join(token for token in tokens)

In [26]:
def eliminar_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

In [27]:
df['Text'] = df['Text'].astype(str)

In [28]:
df['Text'] = df['Text'].apply(clean_text)
df['Text'] = df['Text'].apply(eliminar_stop_words)
df['Text'] = df['Text'].apply(lemmatize)
df['Text'] = df['Text'].apply(eliminar_emoji)

In [29]:
#Agregamos la longitud del texto
df['Longitud_Text'] = df['Text'].str.len()

In [30]:
#Agregamos el número de palabras en el texto
df['Numero_Palabras_Text'] = df['Text'].str.split().map(lambda x: len(x))

In [32]:
#Agregamos el numero de palabras unicas
df['Numero_Palabras_Unicas'] = df['Text'].apply(lambda x: len(set(str(x).split())))

In [35]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(df['Text'])
y = df.Category